In [ ]:
import torch

x = torch.randn(2, 3)
mask = torch.tensor([True, False, True])  # shape (3,)
x.masked_fill(mask, 0)  # ❌ 错，因为 (3,) 无法广播到 (2, 3)

a = torch.rand((3, 5))
print(a)
print(a.unsqueeze(1))
print(a.unsqueeze(1).unsqueeze(-1))

In [ ]:
import torch



N, num_head, q_seq_len, k_seq_len = 1, 2, 5, 5
mul_res = torch.tensor([[
[
    [1.2, 0.3, 0.4, 0.2, 0.1],  # token 0
    [0.3, 1.0, 0.6, 0.5, 0.1],  # token 1
    [0.4, 0.6, 1.3, 0.7, 0.2],  # token 2
    [0.2, 0.5, 0.7, 1.1, 0.4],  # token 3
    [0.1, 0.1, 0.2, 0.4, 1.5]   # token 4 (padding)
],
[
    [0.4, 0.6, 1.3, 0.7, 0.2],  
    [0.2, 0.5, 0.7, 1.1, 0.4],  
    [0.1, 0.1, 0.2, 0.4, 1.5],
    [1.2, 0.3, 0.4, 0.2, 0.1],  
    [0.3, 1.0, 0.6, 0.5, 0.1],  
]
]])
# print(mul_res.shape)    # (N, num_head, q_len, k_len)
key_padding_mask = torch.tensor([[False, False, False, False, True]])  # 只有最后一个 token 是 padding
# print(key_padding_mask)
attn_mask = torch.triu(torch.ones((q_seq_len, k_seq_len)), diagonal=1).bool()   # (q_len=5, k_len=5)

if key_padding_mask is not None:
    key_padding_mask = key_padding_mask.unsqueeze(1).unsqueeze(2)
    key_padding_mask = key_padding_mask.expand((N, num_head, q_seq_len, k_seq_len))
    mul_res = torch.masked_fill(mul_res, key_padding_mask, -torch.inf)
print(mul_res.shape)
print(mul_res)
if attn_mask is not None:
    attn_mask = attn_mask.unsqueeze(0).unsqueeze(0)
    attn_mask = attn_mask.expand((N, num_head, q_seq_len, k_seq_len))
    mul_res = torch.masked_fill(mul_res, attn_mask, -torch.inf)
print(mul_res.shape)
print(mul_res)

# key_padding_mask = key_padding_mask.unsqueeze(1).unsqueeze(2)
# key_padding_mask = key_padding_mask.expand((1, num_head, 5, 5))
# print(key_padding_mask)
# print(torch.softmax(torch.tensor([float("inf"), float("inf")]), dim=0))
# attn_output_weights = attn_output_weights.masked_fill(, float('-inf'))
# print(attn_output_weights)

In [ ]:
from collections import OrderedDict
from transformers import AutoModel
from transformers import BertModel

ckp = "google-bert/bert-base-uncased"

bert = AutoModel.from_pretrained(ckp)
print(bert)

In [5]:
import torch
import torch.nn.functional as F
import torch.nn as nn

inputs = torch.randn((2, 3))
inputs = torch.softmax(inputs, dim=0)
labels = torch.tensor([0, 2])
print(nn.CrossEntropyLoss()(inputs, labels))
print(F.cross_entropy(inputs, labels))
print(F.cross_entropy(inputs[0], labels[0]))

print(inputs[:, 0:2].shape)


tensor(0.8358)
tensor(0.8358)
tensor(0.8265)
torch.Size([2, 2])


In [2]:
import torch
import torch.nn as nn

# t = torch.arange(24).view(2, 3, 4)
# print(t)
# t = t.view(2, 4, 3)
# print(t)


a = torch.arange(12)
print(torch.stack([a] * 3, dim=0))

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11]])


In [1]:
import torch

def top_p(data: torch.Tensor, p):
    max_k = data.shape[-1]
    k = 2000
    top_k, indices = torch.topk(data, k)    # top_k has been sorted by descending order
    cumsum = torch.cumsum(top_k, dim=-1)
    if not torch.all(cumsum[:, -1] > p).item():     # if k is not large enough to cover cumulated probability p 
        top_k, indices = torch.topk(data, max_k)
        cumsum = torch.cumsum(top_k, dim=-1)
    sorted_cum_topk, sorted_cum_indices = torch.sort(cumsum, dim=-1, descending=True) 
    targets = torch.searchsorted(sorted_cum_topk, p)
    bs, max_len = sorted_cum_topk.shape
    positions = torch.arange(max_len).unsqueeze(0).expand(bs, -1)
    top_k = top_k.masked_fill(positions > targets, 0)
    samples_idx = torch.multinomial(top_k, 1)
    res_indices = torch.gather(indices, -1, samples_idx)
    res_values = torch.gather(data, -1, res_indices)
    return res_values, res_indices


t = torch.randn((2, 10))
v, i = top_p(t, 0.6)
# print(t)
# print(v)
# print(i)



RuntimeError: selected index k out of range